# Importação e verificação da quantidade de linhas

In [18]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None #Não limita a exibição das colunas dos DataFrames

df1 = pd.read_csv("TabelaCatalogo49.csv", low_memory=False)
df2 = pd.read_csv("TabelaGeralBd.csv", low_memory=False)

print("Quantidades de linhas no df1:",len(df1))
print("Quantidades de linhas no df1:",len(df2))

Quantidades de linhas no df_1: 4891
Quantidades de linhas no df_2: 35752


# Verificação dos rótulos 

In [19]:
videodf1 = ['Vídeo Comercial', 'Vídeo Institucional', 'video institucional', 'Vídeo Instiutcional', 'Vídeo Terceiros']
videodf2 = ['Vídeo Comercial', 'Vídeo Institucional']

print("Rótulo dos videos da df1:", videodf1)
print('------------')
print("Rótulo dos videos da df2:", videodf2)

Rótulo dos videos da df1: ['Vídeo Comercial', 'Vídeo Institucional', 'video institucional', 'Vídeo Instiutcional', 'Vídeo Terceiros']
------------
Rótulo dos videos da df2: ['Vídeo Comercial', 'Vídeo Institucional']


# Deletar linhas 

Para melhor analisar o arquivo, selecionei apenas as linhas nas quais continham os rótulos que fiquei responsável.

In [23]:
def deleteRow(df, col, lista):
    '''(DataFrame, str, list) -> DataFrame
    Recebe uma DataFrame, o nome da coluna em string e uma lista de rótulos
    da coluna. Deleta todas as linhas cujos valores não estão na lista de
    rótulos. Retorna uma cópia do DataFrame modificado.
    '''
    i = 0
    df_copy = df.copy()
    for item in df[col]:
        if item not in lista:
            df_copy = df_copy.drop(df.index[[i]])
        i += 1
    return df_copy

## Aplicação

In [24]:
col = 'suporte'
df1_cp = deleteRow(df1, col, videodf1)
df2_cp = deleteRow(df2, col, videodf2)
print("Quantidades de linhas que contém vídeo no df1_cp:", len(df1_cp))
print("Quantidades de linhas que contém vídeo no df2_cp:", len(df2_cp))

Quantidades de linhas no df1_cp: 728
Quantidades de linhas no df2_cp: 492


Como o Pandas elimina as linhas e **não** reseta o índice, é importante realizar este processo agora:

In [55]:
df1_cp = df1_cp.reset_index(drop=True)
df2_cp = df2_cp.reset_index(drop=True)

# Verificação das colunas: todos os caracteres são números?

In [28]:
def isFloat(string):
    '''(str) -> bool
    Recebe uma string e verifica se é um número. Se for, retorna True,
    do contrário, retorna False.
    '''
    try:
        float(string)
        return True
    except ValueError:
        return False

In [29]:
def colValuesNumber(df, col):
    '''(DataFrame, str) -> list
    Recebe um DataFrame, o nome da coluna em string e retorna uma lista
    dos valores da coluna que não são números. Se todos os valores forem
    números, o programa imprime um aviso e retorna None.
    Verifica também se esta coluna possui números float ou não.
    '''
    lista_nao_nums = []
    flag = False
    for item in df[col]:
        if not str(item).isnumeric() and not isFloat(str(item)):
            lista_nao_nums += [item]
        elif isFloat(str(item)):
            flag = True
    if lista_nao_nums == []:
        if flag:
            print('Opa, essa coluna tem floats.')
        print('Todas as entradas são números.')
        return None
    return lista_nao_nums

## Aplicação

In [30]:
col = 'titulo'
naonumeros = colValuesNumber(df2_cp, col)
if naonumeros != None:
    norep = []
    # list comprehension que possui a mesma função que .unique()
    [norep.append(num)for num in naonumeros if num not in norep]
    if len(norep) > 100:
        print(f'A coluna {col} tem muitos não-números. Alguns deles são:', norep[:11])
    else:
        print(f'Os não-números da coluna {col} são: ', norep)

A coluna titulo tem muitos não-números. Alguns deles são: ['Viva, México!', 'Cristovão Colombo (Parte II) (*)', 'Xingu (*)', 'Estado de Sítio', 'Cabra Marcado para Morrer', 'Hora da Estrela (A)', 'Walker (*)', 'Americas Preview - Programs 4 & 5', 'Retrato de Tereza', 'Zapotecas y Mistecas: Pueblos de las Nubes', 'Que é Isso Companheiro? (O)']


# Normalização e exportação - df1

In [42]:
def normalizacao(df, coluna, n, lista):
    '''(DataFrame, str, int, list) -> DataFrame
    Recebe um DataFrame, o nome da coluna em string, o comprimento do DataFrame e 
    uma lista de listas cujo primeiro item da sublista sempre será o valor a ser
    substituído.
    '''
    df_cp = df.copy()
    flag = False
    for i in range(n):
        for j in range(len(lista)):
            if df_cp.loc[i, coluna] in lista[j]:
                df_cp.loc[i, coluna] = lista[j][0]
                flag = True
        if flag == False:
            df_cp.loc[i, coluna] = None
        flag = False
    return df_cp 

## Aplicação

### df1

In [57]:
imagem = [['Colorido', 'colorido'], ['Preto e Branco','P&B','p x b', 'Preto/Branco'], ['Colorido e Preto e Branco', 'Cores e P&B', 'Colorido e P&B', 'P&B/Colorido', 'PxB/Colorido']]
legenda = [['Não', 'não', 'Dublado'], ['Sim', 'Sim - inglês', 'Português', 'espanhol']]
versao = [['Original'], ['Cópia'], ['Restaurado', 'original restaurada', 'original restaurado - sem cortes']]
tipo_video = [['Comercial', 'Vídeo Comercial'], ['Institucional', 'Vídeo Institucional', 'video institucional', 'Vídeo Instiutcional'], ['Terceiros', 'Vídeo Terceiros']]

n = len(df1_cp)
df1_cp = normalizacao(df1_cp, 'Versão', n, versao)
df1_cp = normalizacao(df1_cp, 'Imagem', n, imagem)
df1_cp = normalizacao(df1_cp, 'Legenda', n, legenda)
df1_cp = normalizacao(df1_cp, 'suporte', n, tipo_video)

print(df1_cp)

                                         especificação nreferencia  numero  \
0    Walker. Sinopse: Em 1855, o médico, advogado e...          23    23.0   
1    Viva, México!. Sinopse: Painel da vida dos cam...          25    25.0   
2    Cristovão Colombo (Parte II). Sinopse: A saga ...          26    26.0   
3    Xingu. Sinopse: Trabalho realizado pela TV. Ma...          27    27.0   
4    Estado de Sítio. Sinopse: Dan Mitrione é uma m...          28    28.0   
..                                                 ...         ...     ...   
723  De bem com a vida. Sinopse: Drama sensível e b...         477     NaN   
724  Três  Dias do Condor. Sinopse: Classe thriller...         478   478.0   
725  Nunca Te Ví, Sempre Te Amei. Sinopse: Helene H...         481     NaN   
726  A chegada da inglesa Hester Prynne  a Massachu...         479     NaN   
727  1500 liras. Esse é o preço do passaporte para ...         480   480.0   

    região local                            titulo  data tempo 

## Testes unique

In [59]:
print(df1_cp.Imagem.unique())
print(df1_cp.Versão.unique())
print(df1_cp.Legenda.unique())
print(df1_cp.suporte.unique())

['Colorido' 'Preto e Branco' None 'Colorido e Preto e Branco']
[None 'Restaurado']
['Sim' 'Não' None]
['Comercial' 'Institucional' 'Terceiros']


### df2

In [61]:
imagem2 = [['Colorido'], ['Preto e Branco','PxB','p x b', 'preto e branco', 'Preto e branco', 'pXb'], ['Colorido e Preto e Branco', 'cores e p&b', 'colorido e pXb', 'p X b e cores', 'PxB e cores']]
legenda2 = [['Não', 'não'], ['Sim', 'Português', 'espanhol']]
versao2 = [['Original'], ['Cópia'], ['Restaurado']]
tipo_video2 = [['Comercial', 'Vídeo Comercial'], ['Institucional', 'Vídeo Institucional']]

n = len(df2_cp)
df2_cp = normalizacao(df2_cp, 'Imagem', n, imagem2)
df2_cp = normalizacao(df2_cp, 'Legenda', n, legenda2)
df2_cp = normalizacao(df2_cp, 'Versão', n, versao2)
df2_cp = normalizacao(df2_cp, 'suporte', n, tipo_video2)

print(df2_cp)

     LIGAÇÃO  ALUNO (A)  TEMA: GRAU:  ORIENTADOR:  DEPTO:  ÁREA:  \
0         10        NaN    NaN   NaN          NaN     NaN    NaN   
1         11        NaN    NaN   NaN          NaN     NaN    NaN   
2         12        NaN    NaN   NaN          NaN     NaN    NaN   
3         13        NaN    NaN   NaN          NaN     NaN    NaN   
4         29        NaN    NaN   NaN          NaN     NaN    NaN   
..       ...        ...    ...   ...          ...     ...    ...   
487     5823        NaN    NaN   NaN          NaN     NaN    NaN   
488     5824        NaN    NaN   NaN          NaN     NaN    NaN   
489     5825        NaN    NaN   NaN          NaN     NaN    NaN   
490     5826        NaN    NaN   NaN          NaN     NaN    NaN   
491     5827        NaN    NaN   NaN          NaN     NaN    NaN   

     DATA DE DEFESA:  pp  Volumes Nº DE TOMBO  Revisado  Obs:  Encadernação  \
0                NaN NaN      NaN         NaN       NaN   NaN           NaN   
1                NaN NaN 

# Renomear e deletar colunas

## Aplicação 

### df1

In [18]:
# análise das colunas relevantes
df1_col = ['especificação','nreferencia','numero','região','local','titulo','data','tempo','quantidade','produção','rolonum','Bobina','suporte','NúmerodoFilme','País','Gênero','Idioma Original','Legenda','Versão','Sistema','Imagem','Direção','Roteiro','Fotografia','Música','Elenco','Premiações']

In [64]:
df1_cp.columns = ['field_especificacao_j', 'field_referencia_j', 'field_numero_j', 'field_regiao_j', 'field_local_j', 'title', 'field_data_j', 'field_tempo_j', 'field_quantidade_j', 'field_producao_j', 'field_rolonum_j', 'field_bobina_j', 'field_tipo_video_j', 'field_numero_filme_j', 'field_pais_j', 'field_genero_j', 'field_idioma_original_j' , 'field_legenda_j', 'field_versao_j','field_sistema_j', 'field_imagem_j', 'field_direcao_j', 'field_roteiro_j', ' field_fotografia_j', 'field_musica_j', 'field_elenco_j', 'field_premiacao_j']
# Eliminei a coluna Legendadoslide pois esta coluna não possui nenhum valor
df1_cp = df1_cp.drop('Legendadoslide', axis=1)
print(df1_cp)

                                 field_especificacao_j field_referencia_j  \
0    Walker. Sinopse: Em 1855, o médico, advogado e...                 23   
1    Viva, México!. Sinopse: Painel da vida dos cam...                 25   
2    Cristovão Colombo (Parte II). Sinopse: A saga ...                 26   
3    Xingu. Sinopse: Trabalho realizado pela TV. Ma...                 27   
4    Estado de Sítio. Sinopse: Dan Mitrione é uma m...                 28   
..                                                 ...                ...   
723  De bem com a vida. Sinopse: Drama sensível e b...                477   
724  Três  Dias do Condor. Sinopse: Classe thriller...                478   
725  Nunca Te Ví, Sempre Te Amei. Sinopse: Helene H...                481   
726  A chegada da inglesa Hester Prynne  a Massachu...                479   
727  1500 liras. Esse é o preço do passaporte para ...                480   

     field_numero_j field_regiao_j field_local_j  \
0              23.0    

### df2

In [ ]:
# análise das colunas relevantes
df2col = ['LIGAÇÃO','especificação','nreferencia','numero','região','local','titulo','data','tempo','quantidade','produção','Nº do Rolo','Bobina','suporte','NúmerodoFilme','País','Gênero','Idioma Original','Legenda','Versão','Sistema','Imagem','Direção','Roteiro','Fotografia','Música','Elenco','Premiações']

In [67]:
excluir_df2 = ['ALUNO (A)','Legendadoslide','TEMA:','GRAU:','ORIENTADOR:','DEPTO:','ÁREA:','DATA DE DEFESA:','pp','Volumes','Nº DE TOMBO','Revisado','Obs:','Encadernação','Microficha:','Produção da FFLCH:','número','COPIA','Microfilme'] 
df2_cp = df2_cp.drop(excluir_df2, axis=1)
df2_cp.columns = ['field_ligacao_j','field_especificacao_j', 'field_referencia_j', 'field_numero_j', 'field_regiao_j', 'field_local_j', 'title', 'field_data_j', 'field_tempo_j', 'field_quantidade_j', 'field_producao_j', 'field_rolonum_j', 'field_bobina_j', 'field_tipo_video_j', 'field_numero_filme_j', 'field_pais_j', 'field_genero_j', 'field_idioma_original_j' , 'field_legenda_j', 'field_versao_j','field_sistema_j', 'field_imagem_j', 'field_direcao_j', 'field_roteiro_j', ' field_fotografia_j', 'field_musica_j', 'field_elenco_j', 'field_premiacao_j']
print(df2_cp)

     field_ligacao_j                              field_especificacao_j  \
0                 10  Viva, México!. Sinopse: Painel da vida dos cam...   
1                 11  Cristovão Colombo (Parte II). Sinopse: A saga ...   
2                 12  Xingu. Sinopse: Trabalho realizado pela TV. Ma...   
3                 13  Estado de Sítio. Sinopse: Dan Mitrione é uma m...   
4                 29  Cabra Marcado para Morrer. Sinopse: O diretor ...   
..               ...                                                ...   
487             5823  Sessão Solene: Cerimônia de outorga do Título ...   
488             5824  Sessão Solene: Cerimônia de outorga do Título ...   
489             5825  Blow Up - Depois daquele Beijo. Sinopse: Ao am...   
490             5826  Toda Nudez será Castigada. Sinopse: Viúvo se a...   
491             5827  Uma Rua Chamada Pecado. Sinopse: Mulher neurót...   

    field_referencia_j  field_numero_j field_regiao_j field_local_j  \
0                   25      

# Colunas compostas por números: transformar float para int 

In [71]:
# Exemplo
colValuesNumber(df2_cp, 'field_ligacao_j')
print('------------')
colValuesNumber(df2_cp, 'field_numero_j')
print('------------')
colValuesNumber(df2_cp, 'field_quantidade_j')
print('------------')
colValuesNumber(df2_cp, 'field_rolonum_j')
print('------------')
colValuesNumber(df2_cp, 'field_numero_filme_j')

Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.


Por algum motivo, estas colunas possuem números inteiros escritos como float. Sempre que é possível trabalhar com inteiros, em computação, é melhor que assim o seja, pois float é um tipo aproximado e que gera diversas complicações nos programas. Por isso, a tarefa agora é transformar os valores da coluna em strings de inteiros.

In [26]:
def isNumber(string):
    '''(str) -> bool
    Recebe uma string e identifica se é um número. Se sim, retorna True, 
    se não, retorna False.
    '''
    try:
        if string.isnumeric():
            return True
    except AttributeError:
        if isFloat(string):
            return True
    except:
        return False

In [73]:
def floaToInt(df, col):
    '''(DataFrame, str) -> DataFrame
    Recebe um DataFrame e o nome da coluna. Transforma todos os valores que são
    números em inteiros e retorna uma cópia do DataFrame modificado.
    '''
    n = len(df)
    df_cp = df.copy()
    for i in range(n):
        if df_cp.loc[i, col] != np.nan:
            try:
                df_cp.loc[i, col] = str(int(df_cp.loc[i, col]))
            except: # precisei de except-pass para lidar com NaN
                pass
    return df_cp

## Aplicação

### df1

In [80]:
teste = colValuesNumber(df1_cp, 'field_numero_j')
print('------------')
teste1 = colValuesNumber(df1_cp, 'field_quantidade_j') 
print(teste1)
print('------------')
teste2 = colValuesNumber(df1_cp, 'field_rolonum_j')
print(teste2)
print('------------')
teste3 = colValuesNumber(df1_cp, 'field_numero_filme_j')


------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
['3 (Original e Cópia)', '1/4', '2/4', '3/4', '4/4']
------------
['1/2', '2/2', '1/2', '2/2', '2/2', '1/2', '2/2']
------------
Opa, essa coluna tem floats.
Todas as entradas são números.


Como as colunas 'Quantidade' e 'Número do rolo' possuem números passíveis de serem normalizados para inteiros, utilizei o módulo .replace() para substituir estas ocorrências:

In [82]:
# field_quantidade_j
df1_cp['field_quantidade_j'] = df1_cp['field_quantidade_j'].replace(['3 (Original e Cópia)'], '3')
df1_cp['field_quantidade_j'] = df1_cp['field_quantidade_j'].replace(['1/4'], '1')
df1_cp['field_quantidade_j'] = df1_cp['field_quantidade_j'].replace(['2/4'], '2')
df1_cp['field_quantidade_j'] = df1_cp['field_quantidade_j'].replace(['3/4'], '3')
df1_cp['field_quantidade_j'] = df1_cp['field_quantidade_j'].replace(['4/4'], '4')

# field_rolonum_j
df1_cp['field_rolonum_j'] = df1_cp['field_rolonum_j'].replace(['1/2'], '1')
df1_cp['field_rolonum_j'] = df1_cp['field_rolonum_j'].replace(['2/2'], '2')

# verificar se o replace foi aplicado
colValuesNumber(df1_cp, 'field_quantidade_j')
colValuesNumber(df1_cp, 'field_rolonum_j')

Opa, essa coluna tem floats.
Todas as entradas são números.
Opa, essa coluna tem floats.
Todas as entradas são números.


In [85]:
df1_cp = floaToInt(df1_cp, 'field_numero_j') 
df1_cp = floaToInt(df1_cp, 'field_quantidade_j')
df1_cp = floaToInt(df1_cp, 'field_rolonum_j')
df1_cp = floaToInt(df1_cp, 'field_numero_filme_j') 

print(df1_cp)

                                 field_especificacao_j field_referencia_j  \
0    Walker. Sinopse: Em 1855, o médico, advogado e...                 23   
1    Viva, México!. Sinopse: Painel da vida dos cam...                 25   
2    Cristovão Colombo (Parte II). Sinopse: A saga ...                 26   
3    Xingu. Sinopse: Trabalho realizado pela TV. Ma...                 27   
4    Estado de Sítio. Sinopse: Dan Mitrione é uma m...                 28   
..                                                 ...                ...   
723  De bem com a vida. Sinopse: Drama sensível e b...                477   
724  Três  Dias do Condor. Sinopse: Classe thriller...                478   
725  Nunca Te Ví, Sempre Te Amei. Sinopse: Helene H...                481   
726  A chegada da inglesa Hester Prynne  a Massachu...                479   
727  1500 liras. Esse é o preço do passaporte para ...                480   

    field_numero_j field_regiao_j field_local_j  \
0               23      

#### Testes unique

In [87]:
print('Número: ', df1_cp.field_numero_j.unique())
print('--------')
print('Quantidade: ', df1_cp.field_quantidade_j.unique())
print('--------')
print('Número do Rolo: ', df1_cp.field_rolonum_j.unique())
print('--------')
print('Número do filme: ', df1_cp.field_numero_filme_j.unique())

Número:  ['23' '25' '26' '27' '28' '30' '339' '31' '32' '34' '340' '29' '1' '4' '5'
 '7' '8' '9' '10' '11' '3' nan '14' '15' '16' '17' '184' '186' '187' '188'
 '189' '190' '33' '122' '123' '124' '125' '157' '158' '159' '160' '161'
 '162' '163' '164' '165' '166' '167' '168' '169' '170' '171' '172' '173'
 '174' '175' '179' '180' '176' '177' '0' '39' '126' '127' '18' '19' '20'
 '21' '22' '35' '37' '40' '42' '43' '45' '46' '36' '47' '49' '50' '51'
 '53' '56' '57' '58' '155' '2' '60' '61' '62' '63' '64' '65' '67' '68'
 '69' '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '82' '83'
 '84' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98'
 '99' '100' '101' '102' '104' '105' '106' '107' '108' '109' '110' '111'
 '112' '113' '114' '115' '116' '117' '118' '119' '121' '129' '130' '131'
 '132' '133' '134' '136' '137' '138' '139' '140' '141' '142' '143' '144'
 '145' '146' '147' '148' '149' '150' '151' '152' '154' '182' '191' '192'
 '193' '194' '195' '196' '197' '198' '199' '200'

### df2

In [83]:
colValuesNumber(df2_cp, 'field_ligacao_j')
print('------------')
colValuesNumber(df2_cp, 'field_numero_j')
print('------------')
colValuesNumber(df2_cp, 'field_quantidade_j') 
print('------------')
colValuesNumber(df2_cp, 'field_rolonum_j')
print('------------')
colValuesNumber(df2_cp, 'field_numero_filme_j')

Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.
------------
Opa, essa coluna tem floats.
Todas as entradas são números.


In [74]:
df2_cp = floaToInt(df2_cp, 'field_ligacao_j') 
df2_cp = floaToInt(df2_cp, 'field_numero_j') 
df2_cp = floaToInt(df2_cp, 'field_quantidade_j')
df2_cp = floaToInt(df2_cp, 'field_rolonum_j')
df2_cp = floaToInt(df2_cp, 'field_numero_filme_j') 

print(df2_cp)

    field_ligacao_j                              field_especificacao_j  \
0                10  Viva, México!. Sinopse: Painel da vida dos cam...   
1                11  Cristovão Colombo (Parte II). Sinopse: A saga ...   
2                12  Xingu. Sinopse: Trabalho realizado pela TV. Ma...   
3                13  Estado de Sítio. Sinopse: Dan Mitrione é uma m...   
4                29  Cabra Marcado para Morrer. Sinopse: O diretor ...   
..              ...                                                ...   
487            5823  Sessão Solene: Cerimônia de outorga do Título ...   
488            5824  Sessão Solene: Cerimônia de outorga do Título ...   
489            5825  Blow Up - Depois daquele Beijo. Sinopse: Ao am...   
490            5826  Toda Nudez será Castigada. Sinopse: Viúvo se a...   
491            5827  Uma Rua Chamada Pecado. Sinopse: Mulher neurót...   

    field_referencia_j field_numero_j field_regiao_j field_local_j  \
0                   25             25    

#### Testes unique

In [86]:
print('Ligação: ', df2_cp.field_ligacao_j.unique())
print('--------')
print('Número: ', df2_cp.field_numero_j.unique())
print('--------')
print('Quantidade: ', df2_cp.field_quantidade_j.unique())
print('--------')
print('Número do Rolo: ', df2_cp.field_rolonum_j.unique())
print('--------')
print('Número do filme: ', df2_cp.field_numero_filme_j.unique())

Ligação:  ['10' '11' '12' '13' '29' '30' '79' '80' '81' '82' '92' '93' '95' '96'
 '97' '98' '99' '100' '101' '102' '126' '127' '128' '155' '158' '159'
 '237' '238' '239' '240' '275' '276' '277' '351' '411' '412' '413' '414'
 '415' '416' '417' '554' '555' '556' '557' '558' '559' '560' '561' '562'
 '563' '564' '565' '592' '593' '670' '671' '672' '673' '674' '675' '676'
 '687' '766' '767' '788' '791' '792' '793' '794' '795' '796' '797' '798'
 '799' '800' '801' '802' '803' '839' '840' '841' '859' '860' '861' '862'
 '863' '864' '1087' '1088' '1089' '1090' '1091' '1092' '1125' '1126'
 '1127' '1320' '1321' '1322' '1323' '1324' '1413' '1424' '1425' '1426'
 '1427' '1428' '1429' '1430' '1431' '1432' '1433' '1446' '1479' '1480'
 '1481' '1548' '1549' '1573' '1574' '1575' '1576' '1577' '1578' '1579'
 '1584' '1585' '1586' '1587' '1588' '1589' '1590' '1619' '1622' '1763'
 '1764' '1765' '1780' '1781' '1782' '1783' '1784' '1796' '1797' '1798'
 '1799' '1800' '1860' '1875' '1876' '1877' '1878' '1879' '18

# Adição da coluna Ligação à df1

In [93]:
def addLig(df, df1, n, m):
    '''(DataFrame, DataFrame, int, int) -> DataFrame
    Recebe dois DataFrames, o comprimento do primeiro e do segundo DataFrame, 
    respectivamente. 
    Popula a recém criada coluna do primeiro DF "field_ligacao_j" 
    com os valores desta coluna no segundo DF de linhas iguais a partir do critério 
    das colunas "title" e "field_referencia_j".
    '''
    df_cp = df.copy()
    for i in range(n):
        for j in range(m):
            if df.loc[i, 'title'] == df1.loc[j, 'title'] and df.loc[i, 'field_referencia_j'] == df1.loc[j, 'field_referencia_j']:
                df_cp.loc[i, 'field_ligacao_j'] =  df1.loc[j, 'field_ligacao_j']
    return df_cp

## Aplicação

In [96]:
n = len(df1_cp)
m = len(df2_cp)

df1_cp.insert(0, 'field_ligacao_j', None)
df1_cp = addLig(df1_cp, df2_cp, n, m)

print(df1_cp)

    field_ligacao_j                              field_especificacao_j  \
0                79  Walker. Sinopse: Em 1855, o médico, advogado e...   
1                10  Viva, México!. Sinopse: Painel da vida dos cam...   
2                11  Cristovão Colombo (Parte II). Sinopse: A saga ...   
3                12  Xingu. Sinopse: Trabalho realizado pela TV. Ma...   
4                13  Estado de Sítio. Sinopse: Dan Mitrione é uma m...   
..              ...                                                ...   
723            None  De bem com a vida. Sinopse: Drama sensível e b...   
724            None  Três  Dias do Condor. Sinopse: Classe thriller...   
725            None  Nunca Te Ví, Sempre Te Amei. Sinopse: Helene H...   
726            None  A chegada da inglesa Hester Prynne  a Massachu...   
727            None  1500 liras. Esse é o preço do passaporte para ...   

    field_referencia_j field_numero_j field_regiao_j field_local_j  \
0                   23             23    

# Fusão das duas tabelas com .merge()

In [144]:
df = pd.concat([df1_cp, df2_cp])
print(df)
df = pd.merge(df, df2_cp, how='right')
print('--------')
print(df)

    field_ligacao_j                              field_especificacao_j  \
0                79  Walker. Sinopse: Em 1855, o médico, advogado e...   
1                10  Viva, México!. Sinopse: Painel da vida dos cam...   
2                11  Cristovão Colombo (Parte II). Sinopse: A saga ...   
3                12  Xingu. Sinopse: Trabalho realizado pela TV. Ma...   
4                13  Estado de Sítio. Sinopse: Dan Mitrione é uma m...   
..              ...                                                ...   
487            5823  Sessão Solene: Cerimônia de outorga do Título ...   
488            5824  Sessão Solene: Cerimônia de outorga do Título ...   
489            5825  Blow Up - Depois daquele Beijo. Sinopse: Ao am...   
490            5826  Toda Nudez será Castigada. Sinopse: Viúvo se a...   
491            5827  Uma Rua Chamada Pecado. Sinopse: Mulher neurót...   

    field_referencia_j field_numero_j field_regiao_j field_local_j  \
0                   23             23    

# Inserção de títulos provisórios

É importante lembrar que qualquer linha que não possuir 'title' não constará no Drupal. Por isso, e a fim de não perder nenhum item, decidi indicar em 'title' aquelas linhas que, originalmente, não possuem título.

In [146]:
NOTITLE = 'Não há título'

def inserTitle(df, n, col):
    '''(DataFrame, int, str) -> DataFrame
    '''
    df_cp = df.copy()
    for i in range(n):
        if pd.isnull(df.loc[i, col]):
            print('Olá, não tenho título. Meu índice é:', i)
            df_cp.loc[i, col] = NOTITLE
    return df_cp

In [162]:
n = len(df)
df = inserTitle(df, n, 'title')
print(df.title)

0      Americas Preview - Programs 4 & 5
1      Americas Preview - Programs 4 & 5
2       Concurso: Professor Titular. (*)
3       Concurso: Professor Titular. (*)
4                História do Brasil. (*)
                     ...                
640              Nunca Fomos tão Felizes
641                       Sessão Solene.
642      Blow Up - Depois daquele Beijo.
643           Toda Nudez será Castigada.
644               Uma Rua Chamada Pecado
Name: title, Length: 645, dtype: object


# Adição da coluna nid + detalhes

In [166]:
df.insert(0, 'nid', None)
# percebi que, assim como o field_ligacao_j, a coluna field_bobina_j não possui nenhum valor.
df = df.drop('field_bobina_j', axis=1)

In [167]:
def addNid(df):
    df_cp = df.copy()
    j = 46000
    for i in range(len(df)):
        df_cp.loc[i, 'nid'] = j 
        j += 1
    return df_cp

In [168]:
df = addNid(df)
print(df)

       nid field_ligacao_j                              field_especificacao_j  \
0    46000              80                 Americas Preview - Programs 4 & 5.   
1    46001              80                 Americas Preview - Programs 4 & 5.   
2    46002             126  Concurso: Professor Titular Prof. Shozo Motoya...   
3    46003             126  Concurso: Professor Titular Prof. Shozo Motoya...   
4    46004             127  História do Brasil. Fundação Padre Anchieta - ...   
..     ...             ...                                                ...   
640  46640            5814      Nunca Fomos tão Felizes. Dir.: Murilo Salles.   
641  46641            5823  Sessão Solene: Cerimônia de outorga do Título ...   
642  46642            5825  Blow Up - Depois daquele Beijo. Sinopse: Ao am...   
643  46643            5826  Toda Nudez será Castigada. Sinopse: Viúvo se a...   
644  46644            5827  Uma Rua Chamada Pecado. Sinopse: Mulher neurót...   

    field_referencia_j fiel

# Exportação 

In [169]:
df.to_csv('videos_CAPH.csv', index=False)

# Gerenciamento dos campos - Tipo de Conteúdo 

[645 linhas x 28 colunas]  
Nid: $[46000;46600[$   
Rótulos e tipo:

In [158]:
d = {'nid': 'default', 'field_ligacao_j': 'int', 'field_referencia_j': 'text, plain', 'field_numero_j': 'int', 'field_regiao_j':'text, plain', 'field_local_j':'text, plain', 'title':'default', 'field_data_j': 'text, plain', 'field_tempo_j': 'text, plain', 'field_quantidade_j': 'int', 'field_producao_j': 'text, plain', 'field_rolonum_j': 'int', 'field_tipo_video_j': 'str list', 'field_numero_filme_j': 'int', 'field_pais_j': 'text, plain', 'field_genero_j': 'text, plain', 'field_idioma_original_j': 'text, plain', 'field_legenda_j': 'str list', 'field_versao_j': 'str list', 'field_sistema_j': 'text, plain', 'field_imagem_j': 'str list', 'field_direcao_j': 'text, plain', 'field_roteiro_j'
: 'text, plain', 'field_fotografia_j': 'text, plain', 'field_musica_j': 'text, plain', 'field_elenco_j': 'text, plain, 500 chars', 'field_premiacao_j': 'text, plain, 500 chars'}

s = pd.Series(d)

print(s)

nid                                       default
field_ligacao_j                               int
field_referencia_j                    text, plain
field_numero_j                                int
field_regiao_j                        text, plain
field_local_j                         text, plain
title                                     default
field_data_j                          text, plain
field_tempo_j                         text, plain
field_quantidade_j                            int
field_producao_j                      text, plain
field_rolonum_j                               int
field_tipo_video_j                       str list
field_numero_filme_j                          int
field_pais_j                          text, plain
field_genero_j                        text, plain
field_idioma_original_j               text, plain
field_legenda_j                          str list
field_versao_j                           str list
field_sistema_j                       text, plain
